In [1]:
import pandas as pd
import numpy as np

In [4]:
df = pd.read_csv("capture_test.txt")
df

,AcX,AcY,AcZ,GyX,GyY,GyZ,Tmp
0,-350,-636,7304,334,95,-400,-2256
1,1726,-23264,15640,1394,753,-201,-2208
2,-2916,20004,1356,-1206,-1114,353,-2240
3,1070,3440,6050,129,259,-278,-2272
4,-1958,-2244,8208,-134,-541,51,-2240
5,2404,2744,5548,-89,-2,-383,-2208
6,1934,-5064,10090,348,595,-463,-2240
7,-3746,3018,9856,-155,22,191,-2240
8,3062,-3174,8720,-165,-323,97,-2208
9,4426,-3084,8276,102,794,-317,-2240


In [23]:
header = []

acx = ["AcX"+str(i) for i in range(25)]

In [20]:
list_df = []

for i in range(25):
    lst = df.iloc[100*i:100*(i+1),:]
    lst = lst.transpose()
    print(lst)
    list_df.append(lst)

       0      1      2     3     4     5      6     7     8     9   ...  \
AcX  -350   1726  -2916  1070 -1958  2404   1934 -3746  3062  4426  ...   
AcY  -636 -23264  20004  3440 -2244  2744  -5064  3018 -3174 -3084  ...   
AcZ  7304  15640   1356  6050  8208  5548  10090  9856  8720  8276  ...   
GyX   334   1394  -1206   129  -134   -89    348  -155  -165   102  ...   
GyY    95    753  -1114   259  -541    -2    595    22  -323   794  ...   
GyZ  -400   -201    353  -278    51  -383   -463   191    97  -317  ...   
Tmp -2256  -2208  -2240 -2272 -2240 -2208  -2240 -2240 -2208 -2240  ...   

        90     91     92     93     94    95    96    97    98    99  
AcX   -364    152   4970  -5476   3678 -2408 -1152  2934    96 -3260  
AcY  18704 -15728  20256  -7840  -7348 -1014 -3414  -946  2276  2824  
AcZ   1928  11644  -5432  11070  12208  9096  9036  8702  7118  8082  
GyX  -1842   1440  -2557   2773   -949  -207   196  -330   202  -570  
GyY    299     22    136     99   -436  -182